# Calcualte performance rankings from cleaned results

In [1]:
# read cleaned results
import pandas as pd
from pathlib import Path
import pdb, os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

agg_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results_with_default.csv"))
agg_df = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results.csv"))

fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))
fold_df = pd.read_csv(Path("./cleaned_results/tuned_fold_results.csv"))

# make sure output folder exists
output_folder = Path("./performance_rankings")
output_folder.mkdir(exist_ok=True)  

/tmp/ipykernel_1327038/4270952197.py:11: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))


In [2]:
import numpy as np

def get_rank_table(df, metric, table_idx=4):
    """
    NOTE: the df needs to have the following columns defined:
    - {metric}_rank_mean
    - normalized_{metric}__test_mean
    """
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    df = df[df["dataset_name"].isin(DATASETS)]

    overall_ranks = df.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "median", "count"],
            f"normalized_{metric}__test_mean": ["mean", "median"],
            f"normalized_{metric}__test_std": ["mean", "median"],
            f"train_per_1000_inst_mean_{metric}": ["mean", "median"],        
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    # overall_ranks.loc[:, "count"] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)
    overall_ranks.drop(columns=(f"{metric}_rank_mean", "count"), inplace=True)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].apply(lambda x: "{:d}".format(int(x)))

    # mean/median mean-rank
    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "median")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "median")].apply(lambda x: "{:d}".format(int(x)) if int(x) == x else "{:.1f}".format(x))
    
    # normalized metric - mean and std over folds
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "median")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "median")].apply(lambda x: "{:.2f}".format(x))


    # normalized runtime
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "mean")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "median")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "median")].apply(lambda x: "{:.2f}".format(x))
   


    final_table = overall_ranks.set_index("alg_name")

    return final_table


### Save rank tables to file

In [15]:
# save rank tables to csv and latex
from analysis_utils import ALG_DISPLAY_NAMES
table_idx = 4

metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
    "AUC",
]

final_tables = {}

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    agg_df_with_default

    # first with default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df_with_default, metric, table_idx = table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank_with_default.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank_with_default.tex", index=True, escape=False)

    # now without default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df, metric, table_idx=table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank.tex", index=True, escape=False)

/tmp/ipykernel_1327038/2689206630.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '3' '1' '1' '2'
 '7' '1' '1' '1' '3' '1' '8' '5' '5' '12' '5' '4' '6' '5' '4' '4' '8' '6'
 '2' '7' '5' '9' '8' '8' '13' '16' '27']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
/tmp/ipykernel_1327038/2689206630.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['27' '22' '22' '40' '28' '37' '31' '39' '34' '35' '29' '33' '41' '29'
 '35' '31' '36' '38' '31' '37' '37' '34' '37' '45' '40' '36' '40' '38'
 '39' '42' '41' '41' '37' '37' '43' '45' '44' '43' '44' '42' '45' '45'
 '43' '45' '45']' has dtype incompatible w

In [16]:
final_tables['Accuracy']

Accuracy_rank_mean                    \
                              min max   mean median   
alg_name                                              
TabPFN                          1  11   4.83      2   
CatBoost                        1  17   5.03      3   
XGBoost                         1  21   5.18      2   
ResNet                          1  16   6.09      6   
SAINT                           1  21   7.27      6   
LightGBM                        1  18   7.37      7   
FTTransformer                   1  17   8.04      7   
MLP-rtdl                        1  18   8.18    7.5   
NODE                            1  16   8.29    8.5   
RandomForest                    2  19   8.36      8   
DANet                           1  20  10.00      9   
DeepFM                          4  18  10.95     12   
SVM                             1  20  11.21   12.5   
MLP                             3  21  11.47   11.5   
TabNet                          1  23  11.72     12   
TabTransformer                  3  22  12.88     13   
DecisionTree                    3  21  13.39     14   
LinearModel                     4  22  14.00   14.5   
TabFlex                         3  20  14.18   15.5   
STG                             3  22  14.27     15   
KNN                             5  23  15.77     16   
VIME                            6  23  16.35     17   
NAM                             8  23  18.73     20   

               normalized_Accuracy__test_mean         \
                                         mean median   
alg_name                                               
TabPFN                                   0.88   0.95   
CatBoost                                 0.87   0.91   
XGBoost                                  0.88   0.96   
ResNet                                   0.77   0.88   
SAINT                                    0.72   0.86   
LightGBM                                 0.81   0.85   
FTTransformer                            0.71   0.83   
MLP-rtdl                                 0.70   0.76   
NODE                                     0.73   0.77   
RandomForest                             0.74   0.77   
DANet                                    0.71   0.82   
DeepFM                                   0.63   0.71   
SVM                                      0.65   0.65   
MLP                                      0.59   0.57   
TabNet                                   0.58   0.62   
TabTransformer                           0.56   0.62   
DecisionTree                             0.53   0.55   
LinearModel                              0.44   0.39   
TabFlex                                  0.37   0.25   
STG                                      0.48   0.47   
KNN                                      0.37   0.43   
VIME                                     0.34   0.28   
NAM                                      0.29   0.18   

               normalized_Accuracy__test_std         \
                                        mean median   
alg_name                                              
TabPFN                                  0.22   0.23   
CatBoost                                0.17   0.11   
XGBoost                                 0.18   0.11   
ResNet                                  0.20   0.10   
SAINT                                   0.18   0.12   
LightGBM                                0.25   0.15   
FTTransformer                           0.19   0.12   
MLP-rtdl                                0.18   0.08   
NODE                                    0.23   0.14   
RandomForest                            0.18   0.11   
DANet                                   0.23   0.13   
DeepFM                                  0.27   0.25   
SVM                                     0.18   0.13   
MLP                                     0.18   0.10   
TabNet                                  0.23   0.13   
TabTransformer                          0.19   0.18   
DecisionTree                            0.19   0.13   
LinearModel                